# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [4]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [5]:
# Importing the dataset
pokedex = pd.read_csv("pokemon.csv")
pokedex.head(5)
# Turn everying to snake_case
pokedex.columns = list(map(lambda x: x.lower(), pokedex.columns))
pokedex.columns = pokedex.columns.str.replace(' ','_')
# cleaning mega
for i,pokemon in pokedex.iterrows():
    if 'Mega ' in pokemon['name']:
        pokedex['name'][i] = 'Mega ' +pokemon['name'].split(' ',1)[1]

pokedex.head(10)

/var/folders/0l/6vrgp2zn3_151qtvdl8wpjcr0000gn/T/ipykernel_13234/4153211016.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pokedex['name'][i] = 'Mega ' +pokemon['name'].split(' ',1)[1]


,#,name,type_1,type_2,total,hp,attack,defense,sp._atk,sp._def,speed,generation,legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,Mega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False
7,6,Mega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False
8,6,Mega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43,1,False


From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [6]:
# your code here
pokedex['hypothesis'] = pokedex['hp']+pokedex['attack']+pokedex['defense']+pokedex['sp._atk']+pokedex['sp._def']+pokedex['speed']
pokedex
# we can see hypothesis = total

,#,name,type_1,type_2,total,hp,attack,defense,sp._atk,sp._def,speed,generation,legendary,hypothesis
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,318
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,405
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,525
3,3,Mega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False,625
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False,309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True,600
796,719,Mega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True,700
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True,600
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True,680


## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [45]:
# The code we saw in first week
from sklearn.preprocessing import OneHotEncoder
pokedex_cat = pokedex.select_dtypes(object).drop(['name'],axis=1)
pokedex_cat
encoder = OneHotEncoder().fit(pokedex_cat)
encoded = encoder.transform(pokedex_cat).toarray()
onehot_encoded = pd.DataFrame(encoded,columns=encoder.get_feature_names_out(pokedex_cat.columns))

# Now I need to merge type_2 and type_1

# computing unique types
types1 = list(pokedex['type_1'].unique())
types2 = list(pokedex['type_2'].unique())
types = types1.copy()
for type in types2:
    if type not in types and not isinstance(type,float):
        types.append(type)

# creating new onehot
zero_data = np.zeros(shape=(pokedex.shape[0],len(types)),dtype=int)
new_onehot = pd.DataFrame(zero_data,columns=types)

for i,pokemon in pokedex.iterrows():
    new_onehot[pokemon['type_1']][i]=1
    if isinstance(pokemon['type_2'],float):
        pass
    else:
        new_onehot[pokemon['type_2']][i]=1
display(new_onehot)

,Grass,Fire,Water,Bug,Normal,Poison,Electric,Ground,Fairy,Fighting,Psychic,Rock,Ghost,Ice,Dragon,Dark,Steel,Flying
0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
796,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
797,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
798,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0


## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [85]:
poke_y = pokedex['total']


correlation = pd.Series(np.zeros(len(types)),index=types)

for type in types:
    correlation[type]=new_onehot[type].corr(poke_y)
    
correlation.sort_values(axis=0, ascending=False, inplace=True)
correlation

# Dragon 

Dragon      0.229705
Psychic     0.124688
Steel       0.109703
Fire        0.078726
Fighting    0.077786
Ice         0.060248
Flying      0.059383
Dark        0.056154
Rock        0.032731
Electric    0.020971
Ground      0.015060
Ghost       0.003641
Water      -0.021665
Fairy      -0.036698
Grass      -0.052592
Poison     -0.090441
Normal     -0.105331
Bug        -0.145781
dtype: float64

# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [103]:
# your code here
def combining_types(pokemon):
    if isinstance(pokemon['type_2'],float):
        return pokemon['type_1']
    else:
    # I always place first the "bigger" type so that I don't have
        # duplicates (for example Fire Psychic and Psychic Fire)
        if pokemon['type_1']>pokemon['type_2']:
            return pokemon['type_1']+'-'+pokemon['type_2']
        elif pokemon['type_1']<pokemon['type_2']:
            return pokemon['type_2']+'-'+pokemon['type_1']
    
pokedex['combo_type'] = pokedex.apply(combining_types,axis = 1)
pokedex_cat=pokedex['combo_type']
pokedex_cat=pd.DataFrame(pokedex_cat)
encoder = OneHotEncoder().fit(pokedex_cat)
encoded = encoder.transform(pokedex_cat).toarray()
onehot_encoded = pd.DataFrame(encoded,columns=encoder.get_feature_names_out(pokedex_cat.columns))

combs = pokedex['combo_type'].unique()

correlation = pd.Series(np.zeros(len(combs)),index=combs)

for comb in combs:
    correlation[comb]=onehot_encoded['combo_type_'+comb].corr(poke_y)
    
correlation.sort_values(axis=0, ascending=False, inplace=True)
correlation
# The best double combination are Ice-Dragon, Psychi-Dragon and Flying-Dragon.

Ice-Dragon        0.128737
Psychic-Dragon    0.127066
Flying-Dragon     0.120422
Ghost-Dragon      0.102264
Fire-Dragon       0.092660
                    ...   
Poison           -0.064380
Normal-Fairy     -0.069525
Normal-Flying    -0.092626
Poison-Bug       -0.094944
Bug              -0.178699
Length: 133, dtype: float64